In [269]:
from requests import get
from bs4 import BeautifulSoup

In [330]:
#creating loop to create urls for 50 movies at a time

#initializing iterator variable (pages increment in form of 'start=1', then 'start=51', then 'start=101'......)
z = 1

#creating empty list for urls to be stored in
urls = []

#while the number of movies is below 4002 (arbitrary #)
while z < 4002:
    
    #generating url using format to insert the incrementing variable on each loop
    url = 'https://www.imdb.com/search/title/?title_type=feature&moviemeter=,4000&sort=user_rating,desc&start={}&ref_=adv_nxt'.format(z)
    #try and use this url at some point
    #https://www.imdb.com/search/title/?title_type=feature&release_date=1950-01-01,2020-06-30&user_rating=4.0,&start=1&ref_=adv_nxt
    
    #appending the url to the list of urls
    urls.append(url)
    
    #incrementing iterator variable so it can go to the next page (there are 50 movies listed per page)
    z += 50


In [525]:
#for loop to get all the html objects of each array into a soup object

#creating empty soup item to store the generated html files in
html_soups = BeautifulSoup()

#iterating through the url pages
for url in urls:
    
    #creating response to use in BeautifulSoup function
    response = get(url)
    
    #using the BeautifulSoup function on each response, appending the resulting soup object into the empty 'html_soups'
    html_soups.append(BeautifulSoup(response.text, 'html.parser'))

In [526]:
#creating containers for each movie, seperating at 'lister-item mode-advanced' on html doc
m_containers = html_soups.find_all('div', class_='lister-item mode-advanced')

#printing to make sure the results are what we expect; if you go to the URL, there are only 2758 items on the list created, all are here
print(type(m_containers))
print(len(m_containers))

<class 'bs4.element.ResultSet'>
2758


In [527]:


stars = []

In [600]:


for container in m_containers:
    
    #NEED TO FIGURE THIS OUT!!! WHAT IS THE PROBLEM WHY ARENT THE STARS ABLE TO BE ACCCESSED!!????!?!?!?
    
    #while i < 5:
        
    #
    #for i in range(4):
        #if container.find_all('p')[2].find_all('a')[i+1] is not None:
            #star = container.find_all('p')[2].find_all('a')[i+1].text
            #print(star)
        
    #star = container.find_all('p')[2].find_all('a')[1].text
    #print(star)
    #star2 = container.find_all('p')[2].find_all('a')[2].text
    #print(star2)
        #i+=1
    #stars.append(star)
    #stars.append(star2)
#print(stars)

In [604]:
#creating empty arrays to store 
titles = []
years = []
imdb_ratings = []
summs = []
mpaa_rates = []
#creating empty list to store dictionaries of each movie info
movies = []

In [605]:
#for loop to parse through each movie in m_containers and get the info
for container in m_containers:
    
    #finding title of movie through its location in html doc
    title = container.h3.a.text
    
    #appending title to empty title list
    #titles.append(title)

    
    #finding year of movie through its location in html doc
    year = container.h3.find('span', class_='lister-item-year').text
    
    #appending year to empty year list
    #years.append(year)

    
    #if function created to deal with any movies that dont have ratings
   
    #if there is a rating found in the html
    if container.strong is not None:
        
        #imdb rating stored in i_rate
        i_rate = float(container.strong.text)
        
    #if there is NO imdb rating:
    else:
        
        #set i_rate to 0
        i_rate = 0

    #append the value to list
    #imdb_ratings.append(i_rate)
        
    #finding mpaa rating (G/PG/R etc.) in html
    rating = container.p.span.text
    
    #appending mpaa rating to mpaa_rates list
    #mpaa_rates.append(rating)
    
    #finding director in html
    if container.find_all('p')[2].a is not None:
        dir = container.find_all('p')[2].a.text
    else:
        dir = 'NA'

    #finding summary of movie in html
    summary = container.find_all('p', class_='text-muted')[1].text.strip('\n   ')
    
    #appending summary to summs list
    #summs.append(summary)
    
   
    
    #appending the values for movie to dictionaries, creating new 'movie' class for each movie
    movies.append({'movie':{'Title':title,'Year':year, 'IMDB Rating':i_rate, "Summary": summary, "MPAA Rating":rating, "Director":dir}})

In [606]:
movies[:3]

[{'movie': {'Title': 'The Shawshank Redemption',
   'Year': '(1994)',
   'IMDB Rating': 9.3,
   'Summary': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
   'MPAA Rating': 'R',
   'Director': 'Frank Darabont'}},
 {'movie': {'Title': 'The Godfather',
   'Year': '(1972)',
   'IMDB Rating': 9.2,
   'Summary': 'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.',
   'MPAA Rating': 'R',
   'Director': 'Francis Ford Coppola'}},
 {'movie': {'Title': 'The Dark Knight',
   'Year': '(2008)',
   'IMDB Rating': 9.0,
   'Summary': 'When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.',
   'MPAA Rating': 'PG-13',
   'Director': 'Christopher Nolan'}}]

In [367]:
#checking if all le
print(len(titles))
print(len(summs))
print(len(years))
print(len(imdb_ratings))
print(len(mpaa_rates))

2758
2758
2758
2758
2758


To do:
1. Fix the stars section, how to add them to dict

2. find a way to get the documents transferred to mongodb

3. Begin process of creating functionality - research how to do the things we want to do

4. Optimize speed - could the m_containers loop be inside of the "for url in urls" loop? that may make the operation faster

In [415]:
import pandas as pd
df = pd.DataFrame({'movie': titles,
'year': years,
'imdb': imdb_ratings,
'summary': summs,
'mpaa_rates': mpaa_rates
})

#removing movies with 0's as they arent out yet...
#df = df[df.imdb != 0]


In [369]:
titles_years = [{"Title": t, "Year": s} for t, s in zip(titles, years)]
titles_years[:11]

[{'Title': 'The Shawshank Redemption', 'Year': '(1994)'},
 {'Title': 'The Godfather', 'Year': '(1972)'},
 {'Title': 'The Dark Knight', 'Year': '(2008)'},
 {'Title': 'The Godfather: Part II', 'Year': '(1974)'},
 {'Title': 'The Lord of the Rings: The Return of the King', 'Year': '(2003)'},
 {'Title': 'Pulp Fiction', 'Year': '(1994)'},
 {'Title': "Schindler's List", 'Year': '(1993)'},
 {'Title': '12 Angry Men', 'Year': '(1957)'},
 {'Title': 'Inception', 'Year': '(2010)'},
 {'Title': 'Fight Club', 'Year': '(1999)'},
 {'Title': 'The Lord of the Rings: The Fellowship of the Ring',
  'Year': '(2001)'}]

In [416]:
import json
print(json.dumps(movies))

[{"movie": {"Title": "The Shawshank Redemption", "Year": "(1994)", "IMDB Rating": 9.3, "Summary": "Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.", "MPAA Rating": "R"}}, {"movie": {"Title": "The Godfather", "Year": "(1972)", "IMDB Rating": 9.2, "Summary": "The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.", "MPAA Rating": "R"}}, {"movie": {"Title": "The Dark Knight", "Year": "(2008)", "IMDB Rating": 9.0, "Summary": "When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.", "MPAA Rating": "PG-13"}}, {"movie": {"Title": "The Godfather: Part II", "Year": "(1974)", "IMDB Rating": 9.0, "Summary": "The early life and career of Vito Corleone in 1920s New York City is portrayed, while his son, Michael, expands and tightens his